In [12]:
# Misc and Fundamentals
import pandas as pd
import matplotlib.pyplot as plt
import os
import joblib

# Setup 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Models
from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
import statsmodels.api as sm

# Evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [13]:
tv_df = pd.read_csv("../Data/data/streaming_titles_clean.csv")
tv_df.head(3)

,index,show_id,type,title,director,cast,country,date_added,release_year,rating,...,genre.Coming_of_Age,genre.Anthology,genre.Buddy,genre.Parody,genre.Spy/Espionage,genre.Survival,genre.Soap_Opera_/_Melodrama,genre.Dance,genre.Medical,genre.Disaster
0,1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,...,False,False,False,False,False,False,False,False,False,False
1,2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,...,False,False,False,False,False,False,False,False,False,False
2,3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,...,False,False,False,False,False,False,False,False,False,False
